<a href="https://colab.research.google.com/github/marixko/Supervised_Learning_Tutorial/blob/master/The_Basics_of_Supervised_Learning_For_Astronomers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###**About Google's Colaboratory: **

This is a free Jupyter environment that runs in Google's cloud, which means you can run codes in your computer without having to install anything. You can create a copy of this tutorial in your own Google's Drive and make your own changes. Colaboratory also allows you to easily share your code with others! [Read more](https://colab.research.google.com/notebooks/basic_features_overview.ipynb)



---

# Introduction


> **Author**: Lilianne M. I. Nakazono (email: lilianne.nakazono@usp.br) 

> PhD student at Instituto de Astronomia, Geofísica e Ciências Atmosféricas -- Universidade de São Paulo (IAG-USP). Bachelor's degree in Statistics (IME-USP) and in Astronomy (IAG-USP). 

> **April 2019**

---










###**What is Machine Learning?**

From SAS: 

>> *"Machine learning is a method of data analysis that automates analytical model building. It is a branch of artificial intelligence based on the idea that systems can learn from data, identify patterns and make decisions with minimal human intervention."*

###**What is Supervised Learning?**#

From S.B. Kotsiantis (2007): 

>> *"Every instance in any dataset used by machine learning algorithms is represented using the same set of features. The features may be continuous, categorical or binary. If instances are given with known labels (the corresponding correct outputs) then the learning is called *supervised*, in contrast to *unsupervised learning*, where instances are unlabeled."*



---


###**STAR/GALAXY separation**#

In this tutorial we will perform a STAR/GALAXY separation using a real dataset from [S-PLUS](http://www.splus.iag.usp.br/). This data were already matched with [SDSS](https://www.sdss.org/) (DR15) spectroscopical data and it will be used to train and test the supervised classifiers. The final step (not included in this tutorial) is to use the trained model to predict the classification of your unknown objects.
 
 This tutorial will be entirely in Python 3 and we will go through the following topics:
- Introduction to `Pandas` ([Documentation](https://pandas.pydata.org/))
- Data visualization with `seaborn` ([Documentation](https://seaborn.pydata.org/))
- Classification methods with `sklearn` ([Documentation](https://scikit-learn.org/stable/index.html))

---



**Additional information about the data**



ID        -            Object ID Number

RA         -           Right Ascension in decimal degrees [J2000]

Dec          -         Declination in decimal degrees  [J2000]

FWHM_n       -         Normalized Full width at half maximum to detection image seeing (pixels)

 A        -             Profile RMS along major axis (pixels)
 
B         -            Profile RMS along minor axis (pixels)

KrRadDet      -        Kron apertures in units of A or B (pixels)

uJAVA_auto, F378_auto, F395_auto, F410_auto, F430_auto, g_auto, F515_auto, r_auto, F660_auto, i_auto, F861_auto, z_auto      -          Total-restricted magnitudes  (AB) in corresponding filters

class - Spectroscopic classification from SDSS


#**1. Libraries and Functions**


In [0]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import itertools
from mlxtend.plotting import plot_decision_regions
import matplotlib as mpl 
import matplotlib.gridspec as gridspec
from sklearn import metrics

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [0]:
# Modified from: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.3f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

#**2. Read Data**

For statistical and machine learning purposes it is **always**  better to read the data in a dataframe (data structured in labels for rows and columns) format.


In [0]:
#Reading dataset from github and saving as dataframe
url = 'https://raw.githubusercontent.com/marixko/'
file = 'tutorial_classifiers/master/tutorial_data.txt'
df = pd.read_csv(url+file, delim_whitespace=True, low_memory=False)

In [0]:
# Run this cell to quickly check your dataset
df

In [0]:
# Check header
list(df)

#**3. Pre-analysis**



Before applying any kind of analysis, you need to be aware of any problem in your dataset that can affect your training (e.g. missing values and outliers). Sometimes it will require pre-processing your dataset beforehand (e.g. for missing values, interpolating values or removing them from data may be necessary). 

In [0]:
# You can check your dataset by using describe(). 
# It will return the total count, mean, standard deviation,
# minimum, Q1, Q2 (median), Q3 and maximum

df.describe()

# If you want to check a specific feature use for instance:

# df.FWHM_n.describe()

Another good practice is to check high correlations in your dataset, which can allow you to identify which features are redundant. Thus, you can also be able to reduce dimensionality of your dataset.


>> *"The fact that many features depend on one another often unduly influences the accuracy of supervised ML classification models. This problem can be addressed by construction new features from the basic feature set."* -- S.B. Kotsiantis (2007)

(One way to deal with multicollinearity -- when 2 or more features are moderately or highly correlated -- is creating a new feature set using  [Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis).)

In [0]:
plt.close()
f, ax = plt.subplots(figsize=(8, 8))
var = ['FWHM_n', 'A', 'B', 'KrRadDet', 'uJAVA_auto', 
       'F378_auto', 'F395_auto', 'F410_auto', 'g_auto', 'F515_auto',
       'r_auto', 'F660_auto', 'i_auto', 'F861_auto', 'z_auto']
corr = df[var].corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), 
            cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax, center=0, vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

#It would also be interesting to check the correlation plot for each class



Qualitative variables can also be included. In this case, however, there are no qualitative features that came from S-PLUS observations.
But let's check the classification label counts:

In [0]:
# For qualitative variables, use value_counts()
df['class'].value_counts()

Note that for this example the classes are balanced. It represents a best case scenario, which rarely happens in the real world. 

Be very careful with imbalanced datasets! Some methods and metrics are not good for imbalanced cases, some manipulation in your sampling method (e.g. over/under-sampling) or in your algorithm (e.g. penalized classification)  may be necessary. 




> **Note:** Supervised Learning is not suitable for problems like "I want to find very rare objects that we have never found before!". The learning process is based on your ground-truth samples, so you need to ask yourself "Is my ground-truth sample representative of what I want to find?"

#** 4. Feature Selection**

A very important step of the analysis is choosing your input features. Sometimes you already know which features you need to use to achieve your goals, which comes from your previous knowledge about the topic. However, you can also evaluate which features will give you the best performance. We will discuss more about it on the following sections. 

For didactic purposes, let's consider two feature spaces:

> `dim15` = {all useful information from the catalog}

> `dim2` = {normalized FWHM, Profile RMS along major axis}

In [0]:
dim15 = ['FWHM_n', 'A', 'B', 'KrRadDet', 'uJAVA_auto', 
       'F378_auto', 'F395_auto', 'F410_auto', 'g_auto', 'F515_auto',
       'r_auto', 'F660_auto', 'i_auto', 'F861_auto', 'z_auto']

dim2 = ['FWHM_n','A']

#** 5. Sampling training and testing sets **

Regardless of the classification method you choose, you will want to estimate how accurately your predictive model will perform. This is called **cross-validation** and there are several ways to do it. Some examples are:

* **Holdout method**: randomly separate your original dataset into the training and the testing set. It's very common to adopt 1:3 ratio for the size of test/training sets, although you can choose another ratio. Very simple and fast computationally, but you need to be cautious as it is a single run method. Thus, it may be subject to large variabilities

* **Leave-p-out cross-validation**:
Uses p observations as the testing set and the remaining observations as the training set. Repeat to cover any sampling possibility

* **k-fold cross-validation**: the original dataset is randomly partitioned into k equal sized subsamples. One subsample will be used as testing set and the other k-1 as training set. Repeat k times, until each subsample is used exactly once as the testing set.


I strongly recommend that you also check the other methods before choosing one. For this tutorial we will use the **Holdout method**, for simplicity.

In [0]:
label = pd.DataFrame(df['class'])

# Transform strings into numbered labels
label.loc[label['class'] == 'STAR', 'class'] = 0
label.loc[label['class'] == 'GALAXY', 'class'] = 1

# Use train_test_split() to sample your training and testing sets
# Let's fix a random_state=42 in order to have the same sets 
# on each run. Stratify parameter guarantees that the original 
# proportion of the classes is maintained 

X_train, X_test, y_train, y_test = train_test_split(df[dim15], label, 
                                                    test_size=0.3, 
                                                    random_state=42,
                                                   stratify = label)

#** 6. Classification method: Support Vector Machine (SVM)**

We finally reached the point where we are going to run a classification algorithm. It is common to think, at first, that this would be the most complicated part, but a well-done job will require you to spend most of your time on the other steps. 

There are several classification methods you can use, each of them has its own pros and cons, depending on your science goals and on your dataset. I will give you an example using Support Vector Machine (SVM) with linear kernel, but I recommend you to also check other methods (e.g. Random Forest, Logistic Regression, K-NN, ...)

**DON'T FORGET TO:**
 
 - Learn the basic idea of the method. You don't need to know all the math behind it, but you need to know how it works intuitively
 - Check what are the assumptions of the method and if your dataset is in agreement with it
 - Learn what the parameters of your model (a.k.a. hyperparameters) do. Choosing them wisely can be crucial to have good results in the end. Note: the hyperparameters space can also be part of your validation tests

## 6.1. Basic idea

The SVM finds the hyperplane that best separates your data, based on maximizing the margin between each class. For instance, in one dimension SVM will find a point. For two dimensions, it will be a line. For three dimensions, it will be a plane.

To use a linear kernel, we assume that the data is linearly separable. Otherwise, we should use another kernel (e.g. polynomial).


Read more about SVM [here](https://scikit-learn.org/stable/modules/svm.html#scores-probabilities)



## 6.2. Feature space: dim2

In [0]:
# Train your model:
clf2 = SVC(kernel= 'linear')
clf2.fit(X_train[dim2], y_train.values.ravel()) 

# Make the predictions: 
y_pred2 = clf2.predict(X_test[dim2])

# Plot confusion matrix:
matrix = confusion_matrix(y_test['class'], y_pred)
fig = plot_confusion_matrix(matrix, classes=['STAR','GALAXY'])
plt.show()

From the confusion matrix above we can already see how good the results are: most of our stars (galaxies) are being assigned as stars (galaxies) and just a few percent were misclassified.

Now let's check the plot and how the separation looks like:

In [0]:
plt.style.use('seaborn-pastel')
fig = plt.figure(figsize=(18,6))
gs = gridspec.GridSpec(1, 2)
ax = plt.subplot(gs[0,0])
sns.scatterplot(x=X_train.FWHM_n, y=X_train.A, 
                    hue=y_train['class'])

#Calculate margin (from https://scikit-learn.org/stable/auto_examples/svm/plot_svm_margin.html)
w = clf2.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-5, 5)
yy = a * xx - (clf2.intercept_[0]) / w[1]
margin = 1 / np.sqrt(np.sum(clf2.coef_ ** 2))
yy_down = yy - np.sqrt(1 + a ** 2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin

#Plot margin
plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')
plt.xlabel('FWHM_n')
plt.ylabel('A')
plt.xlim(0,8)
plt.ylim(0.8, 10)

plt.title('Training set')


ax = plt.subplot(gs[0,1])
sns.scatterplot(x=X_test.FWHM_n , y=X_test.A, hue=y_test['class'])
plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')
plt.xlim(0,8)
plt.ylim(0.8, 10)
plt.title('Testing set')

plt.show()

The solid line corresponds to the optimal threshold found by SVM. The dashed lines in the plots above correspond to the maximized margin that I mentioned in Section 6.1. 

These are calculated using only a small part of the data: the objects around where the separation may occur, they are called the Support Vectors. Let's check which ones were considered for this classification:

In [0]:
fig = plt.figure(figsize=(9,7))
sns.scatterplot(x=X_train[dim2].FWHM_n, y=X_train[dim2].A, 
                    hue=y_train['class'])

plt.scatter(clf2.support_vectors_[:, 0], 
clf2.support_vectors_[:, 1], s=8, 
zorder=10,color='red', marker='+')

plt.xlim(0.9,2)
plt.ylim(0.8,5)
plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')
plt.title('Support vectors (Training set)')

## 6.3. Feature space: dim15

In the last section we saw how SVM works in a 2D space. In that case, it is possible to visually check the separation. However, we have much more information available. if we analyse them altogether, it can improve our results. Although, it is impossible to visually check the results, so we need to rely on performance metrics that we will discuss further on the next section. 


In [0]:
# Train your model:
clf15 = SVC(kernel= 'linear')
clf15.fit(X_train, y_train.values.ravel())

# Make predictions:
y_pred = clf15.predict(X_test)

# Plot confusion matrix:
matrix = confusion_matrix(y_test['class'], y_pred)
fig = plot_confusion_matrix(matrix, classes=['STAR','GALAXY'])
plt.show()

# Yeah, as simple as that! :) 

#** 7. Validation and Model Selection**

How can we choose between two (or more) different models? 

For that, we have several performance metrics that we can consider when selecting the best model and I will show a few of them.

The way you are going to analyze the metrics depends on your science goals. For instance: 

* In a STAR/GALAXY separation you are probably not interested in a specific class, but in the overall classification. You can evaluate your model using, for example, Accuracy or F-measure

* Suppose you had a STAR/QSO problem instead, where your main goal is to find new QSOs. You can evaluate your model using, for example, Precision, Recall or F-measure. 




## 7.1 Accuracy

Defined as the fraction of correct predictions.

(Note: accuracy will be biased towards the class with higher frequency, don't rely on this measurement if you have an imbalanced dataset)

In [0]:
print("Accuracy")
print(" First model (dim2):", 
      np.round(100*metrics.accuracy_score(y_test, y_pred2),2), '%')
print(" Second model (dim15):", 
      np.round(100*metrics.accuracy_score(y_test, y_pred),2), '%')

## 7.2. Precision

Defined as:

> Precision $\equiv \frac{TP}{(TP+FP)}$

TP - True Positive ; FP - False Positive

Note that you need to define which class will be your "positive". For example:

 
| STAR (predicted) | GALAXY (predicted)
--- | ---
**STAR** (true label) | True Negative | False Positive
**GALAXY** (true label)| False Negative | True Positive


In Astronomy, it's called **purity**.

In [0]:
P2 = metrics.precision_score(y_test, y_pred2, pos_label=1)
P = metrics.precision_score(y_test, y_pred, pos_label=1)

print("Galaxy Precision")
print(" First model (dim2):", np.round(100*P2,2), '%')
print(" Second model (dim15):", np.round(100*P,2), '%')

# Exercise: Calculate star precision for each model

## 7.3. Recall

Defined as:

> Recall $\equiv \frac{TP}{(TP+FN)}$

TP - True Positive ; FN - False Negative

In Astronomy, it's called **completeness**.

In [0]:
R2 = metrics.recall_score(y_test, y_pred2, pos_label=1)
R = metrics.recall_score(y_test, y_pred, pos_label=1)


print("Galaxy Recall")
print(" First model (dim2):", np.round(100*R2,2), '%')
print(" Second model (dim15):", np.round(100*R,2), '%')

# Exercise: Calculate star recall for each model

## 7.4. F-measure

It's the harmonic mean of Precision and Recall:

$F = \frac{1}{2}\Big(P_i^{-1}+R_i^{-1}\Big)^{-1}  = 2 \times \frac{P_iR_i}{P_i+R_i}, F  \in [0,1]$



In [0]:
print("F-measure")
print(" First model  (dim2):", np.round(metrics.f1_score(y_test, y_pred2),3))
print(" Second model  (dim15):", np.round(metrics.f1_score(y_test, y_pred),3))

## Final message

We came to the end of this tutorial, yay! :)

Although it is called "Machine Learning", you are still the one who is going to make crucial decisions. And that is hard work! I hope I was able to give you at least a brief idea of all the steps involved in the process. 

Now, play around with the code:
* Try other algorithms with the same feature selection and compare your results using the performance metrics
* Test changing the parameters of your model
* Try it with your own dataset!







## Read more:

[Supervised Machine Learning: A Review of Classification Techniques](https://books.google.com/books?hl=en&lr=&id=vLiTXDHr_sYC&oi=fnd&pg=PA3&dq=review+supervised+learning&ots=CYpwxt2Bnn&sig=Y79PK3w3Q8CefKaTh03keRFEwyg#v=onepage&q=review%20supervised%20learning&f=false) (S.B. Kotsiantis, 2007)

An Empirical Comparison of Supervised Learning Algorithms Rich (Rich Caruana and Alexandru Niculescu-Mizil, 2006)

Classification of Imbalanced Data: a Review (Yanmin Sun, Andrew K. C. Wong and Mohamed S. Kamel, 2009)


[Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)

 [A Practical Guide to Support Vector Classification](https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf) (Chih-Wei Hsu, Chih-Chung Chang, and Chih-Jen Lin, 2016)



